In [119]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import (cross_val_score, RepeatedStratifiedKFold, GridSearchCV)
from sklearn.metrics import confusion_matrix
from paths import RAW_DIR

In [120]:
data = pd.read_csv(RAW_DIR / 'heart_disease.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   Diabetes              253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  GenHlth               253680 non-null  float64
 13  MentHlth              253680 non-null  float64
 14  PhysHlth              253680 non-null  float64
 15  

In [121]:
# General constants -> TODO uppercase all constants
goal_cost_reduction = 0.8
risk_reduction = 0.25
accept_prob = 0.85
heart_attack_treatment_cost = 50000
plan_cost = 1000

# Proportion of people who suffers a heart attack (estimation).
heart_attack_estimation = len(data[data['HeartDiseaseorAttack'] == 1])

current_cost = heart_attack_estimation * heart_attack_treatment_cost
new_cost = 0.8 * current_cost  

print("Current cost:", current_cost, "€")
print("New cost:", new_cost, "€")
len(data[data['HeartDiseaseorAttack'] == 1]) / len(data)

Current cost: 1194650000 €
New cost: 955720000.0 €


0.09418558814254178

$$
R = 50000 * FN \\
+ 50000 * 0.15 * TP \\
+ 0.85 * (1 - A_p) * TP * 50000 \\
+ 0.85 * (1 - A_p) * TP * 1000 \\
+ 0.85 * (1 - A_p) * FP * 1000 \\
+ 0.85 * A_p * TP * 0.25 * 50000 \\
+ 0.85 * A_p * TP * 0.25 * 1000 \\
+ 0.85 * A_p * FP * 1000 \\
+ 0.85 * A_p * TP * 0.75 * 1000
$$
---
$$
R = 50000 * (FN + 0.15 * TP)
+ 0.85 * 50000 * TP (A_p * 0.25 + 1 - A_p)
+ 0.85 * 1000 * (1 - A_p) * (TP + FP)
+ 0.85 * 1000 * A_p* (FP + TP)
$$
---
$$
R = 50000 * (FN + 0.15 * TP) ;; the ones that have heart attacks but were not offered the plan or didn't accept it
+ 0.85 * 1000 * (TP + FP) ;; the ones that were offered the plan and accepted it
+ 0.85 * 50000 * TP (1 - 0.75 A_p) ;; the ones that accepted the plan, but don't reduce their risk or don't adhere to the plan
$$
---
$$
\frac{(R 
- 50000 * (FN + 0.15 * TP) 
- 0.85 * 1000 * (TP + FP)
- 0.85 * 50000 * TP)}{-0.85 * 50000 * TP * 0.75} = A_p 
$$

### Cases
- no plan(N) = TN + FN: 
  - Case 1: heart attack (not predicted) => FN => 50000

- plan(P) = TP + FP: 
  - not accept => 0.15 (TP + FP)
    - Case 2: heart attack (predicted, not accepted) => 0.15 * TP => 50000
  - accept => 0.85 (TP + FP)
    - not adhere
      - Case 3: heart attack (predicted, accepted, not adhered) => 0.85 (1 - A_p) TP => 51000
      - Case 4: not heart attack (predicted, accepted, not adhered) => 0.85 (1 - A_p) FP => 1000
    - adhere => A_p
      - Case 5: heart attack (predicted, accepted, adhered) => 0.85 * A_p * TP * 0.25 => 51000
      - Case 6: not heart attack (predicted, accepted, adhered) => 0.85 * A_p * FP + 0.85 * A_p * TP * 0.75 => 1000

### Variables
- risk_reduction = 0.75
- accept_prob = 0.85
- adherence_prob = 0 # need to calculate

Following the list above we have 6 different cost cases in the new system
- case_1 = heart_attack_treatment_cost * false_negative
- case_2 = heart_attack_treatment_cost * (1 - accept_prob) * true_positive
- case_3 = heart_attack_treatment_cost + plan_cost * accept_prob * (1 - adherence_prob) * true_positive
- case_4 = plan_cost * accept_prob * (1 - adherence_prob) * false_positive
- case_5 = heart_attack_treatment_cost + plan_cost * accept_prob * adherence_prob * risk_reduction * true_positive
- case_6 = plan_cost * accept_prob * adherence_prob * false_positive + accept_prob * adherence_prob * (1 - risk_reduction) * true_positive

new_cost_formula = case_1 + case_2 + case_3 + case_4 + case_5 + case_6

-> get adherence from: new_cost = 0.8 * old_cost

In [122]:
TARGET = 'HeartDiseaseorAttack'

y = data[TARGET]
X = data.drop(TARGET, axis=1)

In [123]:
discriminant_analysis = LinearDiscriminantAnalysis()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(
    discriminant_analysis, X, y,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1)

print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Mean Accuracy: 0.902 (0.001)


In [124]:
lda_model = discriminant_analysis.fit(X, y)
pred = discriminant_analysis.predict(X)

conf = pd.DataFrame(
  confusion_matrix(y, pred),
  columns = ['Predicted 0', 'Predicted 1'],
  index = ['True 0', 'True 1']
)
print(conf)

        Predicted 0  Predicted 1
True 0       224121         5666
True 1        19146         4747


In [125]:
conf = confusion_matrix(y, pred)
true_negative = conf[0][0]
false_negative = conf[1][0]
false_positive = conf[0][1]
true_positive = conf[1][1]

print(true_negative, false_negative, false_positive, true_positive)


224121 19146 5666 4747


In [126]:
# check if formula is correct and takes into account the whole population
adherence_prob = 0.5 # cte as example

case_1 = false_negative
case_1f = true_negative # this case has no cost
case_2 = (1 - accept_prob) * true_positive
case_2f = (1 - accept_prob) * false_positive # this case has no cost
case_3 = accept_prob * (1 - adherence_prob) * true_positive
case_4 = accept_prob * (1 - adherence_prob) * false_positive
case_5 = accept_prob * adherence_prob * risk_reduction * true_positive
case_6 = accept_prob * adherence_prob * false_positive + accept_prob * adherence_prob * (1 - risk_reduction) * true_positive

new_total_population = case_1 + case_1f + case_2 + case_2f + case_3 + case_4 + case_5 + case_6

print("Original population:", len(data), "\nNew population:", round(new_total_population))
print(len(data) == round(new_total_population))

Original population: 253680 
New population: 253680
True


In [127]:
# Ideal case should test
true_negative = 229787
false_negative = 0
false_positive = 0
true_positive = 23893
adherence_prob = 1 

new_cost = (
  false_negative * heart_attack_treatment_cost +
  (1 - accept_prob) * true_positive * heart_attack_treatment_cost +
  accept_prob * (1 - adherence_prob) * true_positive * (heart_attack_treatment_cost + plan_cost) +
  accept_prob * (1 - adherence_prob) * false_positive * plan_cost +
  accept_prob * adherence_prob * risk_reduction * true_positive * (heart_attack_treatment_cost + plan_cost) + 
  accept_prob * adherence_prob * false_positive * plan_cost +
  accept_prob * adherence_prob * (1 - risk_reduction) * true_positive * plan_cost
  )

print(round(new_cost), "€")
print(new_cost / current_cost, "%")

453369675 €
0.3795 %


In [128]:
# cost if model was perfect and 100% adherence
best_cost = 51000 * heart_attack_estimation * 0.25 * 0.85 + 1000 * heart_attack_estimation * 0.75  * 0.85 + 50000 * 0.15 * heart_attack_estimation 
print(best_cost, "€")
print(best_cost / current_cost, "%")

453369675.0 €
0.3795 %


In [129]:
# Summarized formula
best_cost_ = (  
  heart_attack_treatment_cost * (false_negative + true_positive) + accept_prob * plan_cost * (true_positive + false_positive) +
  adherence_prob * accept_prob * true_positive * heart_attack_treatment_cost * (risk_reduction - 1)
)

print(best_cost_, "€")
print(best_cost_ / current_cost, "%")
best_cost_ - best_cost

453369675.0 €
0.3795 %


0.0

new_cost = current_cost * 0.8 -> get adherence from this assumption. Just to check formula
#
adherence_prob == (current_cost * goal_cost_reduction - heart_attack_treatment_cost * (false_negative + true_positive) - accept_prob * plan_cost * (true_positive + false_positive)) / (accept_prob * true_positive * heart_attack_treatment_cost * (risk_reduction - 1))


In [130]:
print(current_cost, goal_cost_reduction, heart_attack_treatment_cost, plan_cost, accept_prob, risk_reduction, true_positive, false_positive, false_negative)

1194650000 0.8 50000 1000 0.85 0.25 23893 0 0


In [139]:
# model gives us confusion matrix values and the rest are pre-defined constants -> we can get adherence for a given model and fixed goal cost reduction

def get_adherence(current_cost, goal_cost_reduction, heart_attack_treatment_cost, plan_cost, accept_prob, risk_reduction, conf):
  false_negative = conf[1][0]
  false_positive = conf[0][1]
  true_positive = conf[1][1]
  
  return (
  (current_cost * goal_cost_reduction - heart_attack_treatment_cost * (false_negative + true_positive) - accept_prob * plan_cost * (true_positive + false_positive)) / 
  (accept_prob * true_positive * heart_attack_treatment_cost * (risk_reduction - 1))
  )

In [143]:
# Test function
print(current_cost, goal_cost_reduction, heart_attack_treatment_cost, plan_cost, accept_prob, risk_reduction)
test_adherence = 0.3 

discriminant_analysis = LinearDiscriminantAnalysis()
discriminant_analysis.fit(X, y)
pred = discriminant_analysis.predict(X)

conf = confusion_matrix(y, pred)
false_negative = conf[1][0]
false_positive = conf[0][1]
true_positive = conf[1][1]

new_test_cost = (  
  heart_attack_treatment_cost * (false_negative + true_positive) + accept_prob * plan_cost * (true_positive + false_positive) +
  test_adherence * accept_prob * true_positive * heart_attack_treatment_cost * (risk_reduction - 1)
)
test_reduction = new_test_cost / current_cost
new_test_adherence = get_adherence(current_cost, test_reduction, heart_attack_treatment_cost, plan_cost, accept_prob, risk_reduction, conf)

print(test_adherence == new_test_adherence)

1194650000 0.8 50000 1000 0.85 0.25
True
